In [1]:
import wandb
import os
import pandas as pd

os.environ["WANDB_NOTEBOOK_NAME"] = 'EDA.ipynb' #'config["main"]["experiment_name"]'
run = wandb.init(project="nyc_airbnb", group="eda", job_type="EDA", save_code=True)
# wandb.save('bob.ipynb')
# os.environ["WANDB_NOTEBOOK_NAME"] = 'EDA.ipynb' #'config["main"]["experiment_name"]'
local_path = wandb.use_artifact("sample.csv:latest").file()
df = pd.read_csv(local_path)

wandb: Currently logged in as: shawnyazdani. Use `wandb login --relogin` to force relogin


Applying Pandas Profiling to dataset


In [2]:
import pandas_profiling

profile = pandas_profiling.ProfileReport(df)
profile.to_widgets()

C:\Users\shawn\AppData\Local\Temp\ipykernel_33416\2516835558.py:1: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\shawn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ydata_profiling\model\correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'Function <code object pandas_auto_compute at 0x000001F1D31FD970, file "C:\Users\shawn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ydata_profiling\model\pandas\correlations_pandas.py", line 164>')
  warnings.warn(
C:\Users\shawn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\seaborn\matrix.py:260: FutureW

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Takeaways: Prices have some right-skewing outliers, as the price goes as high as $10000, even though the 95th percentile price is ~$350. We'll remove the outlier entries, determined to be those with a price > $350. 

In [3]:

# Drop outliers
min_price = 10
max_price = 350
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()
# Convert last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])


Checking that dates have been converted to date-type datetime, and that the price now has a max of $350.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19001 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              19001 non-null  int64         
 1   name                            18994 non-null  object        
 2   host_id                         19001 non-null  int64         
 3   host_name                       18993 non-null  object        
 4   neighbourhood_group             19001 non-null  object        
 5   neighbourhood                   19001 non-null  object        
 6   latitude                        19001 non-null  float64       
 7   longitude                       19001 non-null  float64       
 8   room_type                       19001 non-null  object        
 9   price                           19001 non-null  int64         
 10  minimum_nights                  19001 non-null  int64         
 11  number_

In [5]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
count,1.900100e+04,1.900100e+04,19001.000000,19001.000000,19001.000000,19001.000000,19001.000000,15243,15243.000000,19001.000000,19001.000000
mean,1.883041e+07,6.639459e+07,40.728063,-73.950827,122.340456,6.906900,23.797747,2018-10-01 08:24:05.384766720,1.380928,6.583811,109.725383
min,2.539000e+03,2.571000e+03,40.508730,-74.239140,10.000000,1.000000,0.000000,2011-05-12 00:00:00,0.010000,1.000000,0.000000
25%,9.355498e+06,7.728754e+06,40.688820,-73.982050,66.000000,1.000000,1.000000,2018-06-29 12:00:00,0.190000,1.000000,0.000000
50%,1.938754e+07,3.048785e+07,40.721710,-73.954630,100.000000,2.000000,6.000000,2019-05-19 00:00:00,0.720000,1.000000,39.000000
75%,2.891952e+07,1.048354e+08,40.763210,-73.934490,160.000000,5.000000,24.000000,2019-06-23 00:00:00,2.010000,2.000000,219.000000
max,3.648561e+07,2.742733e+08,40.913060,-73.717950,350.000000,1250.000000,607.000000,2019-07-08 00:00:00,27.950000,327.000000,365.000000
std,1.096986e+07,7.782663e+07,0.055389,0.046825,71.530346,21.456544,45.493455,NaN,1.689988,31.154750,130.599899


In [6]:
# wandb.save('eda.ipynb')
run.finish()